## Introduction to Linkup

Linkup is a web search engine designed for AI applications. It connects AI systems to the internet through an API that delivers grounding data, enriching outputs and improving precision, accuracy, and factual reliability

## Linkup search engine example

In [ ]:
!pip install linkup-sdk==0.9.0
!pip install sambanova==1.2.0

In [5]:
import os
from linkup import LinkupClient
from sambanova import SambaNova
import json

In [ ]:
os.environ["LINKUP_API_KEY"] = "your-api-key"
os.environ["SAMBANOVA_API_KEY"] = "your-api-key"
# or dotenv.load_dotenv()
linkup_client = LinkupClient()
sambanova_client = SambaNova(
    api_key=os.environ["SAMBANOVA_API_KEY"],
)

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "search_web",
        "description": "Search the web in real time. Use this tool whenever the user needs trusted facts, news, or source-backed information. Returns comprehensive content from the most relevant sources.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query"
                }
            },
            "required": ["query"]
        }
    }
}]

# Ask a question
messages = [
    {"role": "user", "content": "Tell me about SambaNova RDU chips"}
]

response = sambanova_client.chat.completions.create(
    model="gpt-oss-120b",
    messages=messages,
    tools=tools
)

In [ ]:
# Check if Sambanova wants to call our function
if response.choices[0].message.tool_calls:
    # Get the function call details
    tool_call = response.choices[0].message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)
    
    # Call Linkup to search the web
    linkup_response = linkup_client.search(query=args["query"], depth="standard", output_type="sourcedAnswer", structured_output_schema=None)
    search_results = json.dumps(linkup_response.model_dump())
    
    # Add the function call and result to messages
    messages.append(response.choices[0].message)
    messages.append({
        "role": "tool",
        "name": "search_web",
        "content": search_results,
        "tool_call_id": tool_call.id
    })
    
    # Get final response with search results
    final_response = sambanova_client.chat.completions.create(
        model="gpt-oss-120b",
        messages=messages
    )
    
    print(final_response.choices[0].message.content)
else:
    print(response.choices[0].message.content)

**SambaNova Reconfigurable Dataflow Unit (RDU) chips – a quick‑reference guide**  

| Generation | Chip name | Process | Core count | Compute (BF16/FP16) | On‑chip SRAM | HBM | DDR/External memory | Peak power | Notable architectural blocks | Typical system integration |
|------------|----------|---------|------------|----------------------|--------------|------|----------------------|------------|------------------------------|----------------------------|
| **1st‑gen (Cardinal SN10)** | SN10‑RDU | 7 nm (TSMC) | ~800 RDU cores | ~300 TFLOPS BF16 | 8 GB distributed SRAM (≈PB/s) | 0 GB (no HBM) | DDR4/5 up to 1 TB | ~400 W | Pattern Compute Units (PCU), Pattern Memory Units (PMU), 3‑D switch fabric, AGU/CU interconnect | 8‑chip “SN10‑8R” module; used in early DataScale prototypes and Hot‑Chips 33 demo |
| **2nd‑gen (Cardinal SN30)** | SN30‑RDU | 5 nm (TSMC) | ~960 cores | ~400 TFLOPS BF16 | 16 GB SRAM | 32 GB HBM2e | DDR5 up to 2 TB | ~500 W | Larger SRAM banks, first HBM integration, r